In [1]:
# !pip install -qq transformers genomic-benchmarks datasets

In [2]:
# from google.colab import drive

# drive.mount('/content/drive')

In [3]:
from pathlib import Path

### Parameters
MODEL_NAME = "davidcechak/DNADebertaK8"
TOKENIZER_NAME = "armheb/DNA_bert_6"
K = 8
STRIDE = 1

# if less than 1, only this fraction of each dataset is used
DATASET_THINING = 1

# BENCHMARKS_FOLDER = '/root/.genomic_benchmarks' # for google colab
BENCHMARKS_FOLDER = '/home/jovyan/.genomic_benchmarks/' #(for INFRA HUB)

# for long-sequence datasets:
DATASETS = [('human_enhancers_cohn', 0), ('human_ocr_ensembl', 0), ('human_enhancers_ensembl', 0)]
# "Token indices sequence length is longer than the specified maximum sequence length for this model (517 > 512). Running this sequence through the model will result in indexing errors"
# ('human_enhancers_cohn', 0), ('human_ensembl_regulatory', 0), ('human_ocr_ensembl', 0), ('human_enhancers_ensembl', 0)

# short-sequence datasets
# DATASETS = [('demo_coding_vs_intergenomic_seqs', 0),
#  ('demo_human_or_worm', 0), ('human_nontata_promoters', 0)]

# All datasets
# DATASETS = [('demo_coding_vs_intergenomic_seqs', 0),
#  ('demo_human_or_worm', 0), ('human_enhancers_cohn', 0), ('human_enhancers_ensembl', 0),
#  ('human_ensembl_regulatory', 0), ('human_nontata_promoters', 0), ('human_ocr_ensembl', 0)]


# if ensemble refuses connection - "[Errno 104] Connection reset by peer", use attribute use_cloud_cache=True
USE_CLOUD_CACHE = True

BATCH_SIZE = 32
LEARNING_RATE = 8e-5
EPOCHS = 1
RUNS = 1

# do not forget to attach drive
# OUTPUT_PATH = 'drive/MyDrive/genomic_benchmarks/experiment1.csv'
OUTPUT_PATH = Path.cwd() / Path('experiment1.csv')
print(OUTPUT_PATH)

print(DATASETS)

/home/jovyan/cDNA-pretraining/experiments/genomic_benchmarks/experiment1.csv
[('human_enhancers_cohn', 0), ('human_ocr_ensembl', 0), ('human_enhancers_ensembl', 0)]


In [4]:
from itertools import product
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    TOKENIZER_NAME, 
    model_max_length=512,
    # truncation=True, #the method does not support truncation specification
    # max_length=510
)

alphabet = ('A', 'C', 'T', 'G')
vocab = list(map(''.join, product(alphabet, repeat=K)))

tokenizer.add_tokens(vocab)

65536

In [5]:
text = ""
for i in range(600):
    text+=str('a ')
print(len(text))
# tokenizer([text], truncation =True, max_length = 512)


1200


## Download benchmark datasets and tokenizer

In [6]:
from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check.info import is_downloaded
from pathlib import Path
from tqdm.autonotebook import tqdm

for dataset_name, dataset_version in tqdm(DATASETS):
    if not is_downloaded(dataset_name):
        download_dataset(dataset_name, version=dataset_version, use_cloud_cache=USE_CLOUD_CACHE)

benchmark_root = Path(BENCHMARKS_FOLDER)

  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
def kmers_strideK(s, k=K):
    return [s[i:i + k] for i in range(0, len(s), k) if i + k <= len(s)]

def kmers_stride1(s, k=K):
    return [s[i:i + k] for i in range(0, len(s)-k+1)]

if (STRIDE == 1):
  kmers = kmers_stride1
else:
  kmers = kmers_strideK

# function used for the actual tokenization
def tok_func(x): return tokenizer(" ".join(kmers(x["seq"])), truncation=True)

# example
example = tok_func({'seq': 'ATGGAAAGAGGCACCATTCT'})
print(example)
tokenizer.decode(example['input_ids'])

{'input_ids': [2, 16136, 52241, 65592, 53460, 4930, 7417, 17366, 57162, 19737, 66647, 57679, 21806, 9387, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


'[CLS] ATGGAAAG TGGAAAGA GGAAAGAG GAAAGAGG AAAGAGGC AAGAGGCA AGAGGCAC GAGGCACC AGGCACCA GGCACCAT GCACCATT CACCATTC ACCATTCT [SEP]'

## Looping through datasets, fine-tuning the model for each of them, logging metrics

In [8]:
import pandas as pd
import numpy as np
from random import random, randrange
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset, DatasetDict, load_metric

def compute_metrics_binary(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

def compute_metrics_multi(eval_preds):
    metric = load_metric("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

outputs = []

for dataset_name, dataset_version in tqdm(DATASETS):
    

    labels = sorted([x.stem for x in (benchmark_root / dataset_name / 'train').iterdir()])

    tmp_dict = {}

    for split in ['train', 'test']:
        for nlabel, label in enumerate(labels):
            for f in (benchmark_root / dataset_name / split / label).glob('*.txt'):
                txt = f.read_text()
                if not DATASET_THINING or DATASET_THINING==1:
                    tmp_dict[f"{label} {f.stem}"] = (split, nlabel, txt)
                elif random() < DATASET_THINING:
                    tmp_dict[f"{label} {f.stem}"] = (split, nlabel, txt)

    df = pd.DataFrame.from_dict(tmp_dict).T.rename(columns = {0: "dset", 1: "cat", 2: "seq"})

    ds = Dataset.from_pandas(df)

    tok_ds = ds.map(tok_func, batched=False, remove_columns=['__index_level_0__', 'seq'])
    tok_ds = tok_ds.rename_columns({'cat':'labels'})

    dds = DatasetDict({
        'train': tok_ds.filter(lambda x: x["dset"] == "train").remove_columns('dset'),
        'test':  tok_ds.filter(lambda x: x["dset"] == "test").remove_columns('dset')
    })

    compute_metrics = compute_metrics_binary if len(labels) == 2 else compute_metrics_multi

    for _ in range(RUNS):

        model_cls = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(labels))

        args = TrainingArguments('outputs', learning_rate=LEARNING_RATE, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
            evaluation_strategy="epoch", per_device_train_batch_size=BATCH_SIZE, per_device_eval_batch_size=BATCH_SIZE*2,
            num_train_epochs=EPOCHS, weight_decay=0.01, save_steps=100000, seed=randrange(1,10001), report_to='none')
        
        trainer = Trainer(
            model_cls, 
            args, 
            train_dataset=dds['train'], 
            eval_dataset=dds['test'],
            tokenizer=tokenizer, 
            compute_metrics=compute_metrics
        )
        trainer.train()
        
        max_accuracy = max([x['eval_accuracy'] for x in trainer.state.log_history if 'eval_accuracy' in x])
        max_f1 = max([x['eval_f1'] for x in trainer.state.log_history if 'eval_f1' in x]) if len(labels) == 2 else np.nan
        train_runtime = max([x['train_runtime'] for x in trainer.state.log_history if 'train_runtime' in x])
        
        outputs.append((dataset_name, max_accuracy, max_f1, train_runtime))




  0%|          | 0/3 [00:00<?, ?it/s]

Parameter 'function'=<function tok_func at 0x7f99e64fc4c0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/27791 [00:00<?, ?ex/s]

  0%|          | 0/28 [00:00<?, ?ba/s]

  0%|          | 0/28 [00:00<?, ?ba/s]

...

  0%|          | 0/124 [00:00<?, ?ba/s]

  0%|          | 0/124 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/davidcechak/DNADebertaK8/resolve/main/config.json from cache at /home/jovyan/.cache/huggingface/transformers/808901853890ddb7e1ac8c4ecf5112272702596ee51b69cdf389c63da11fe2e9.b628e4d5e7f344695b21fd7ca542aa0f8188268bc6c6b3380dc1e19a16c2447e
Model config DebertaConfig {
  "_name_or_path": "davidcechak/DNADebertaK8",
  "architectures": [
    "DebertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": null,
  "position_biased_input": true,
  "relative_attention": false,
  "torch_dtype": "float32",
  "transformers_

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.314700,0.298885,0.873135,0.873580


***** Running Evaluation *****
  Num examples = 30970
  Batch size = 64


Training completed. Do not forget to share your model on huggingface.co/models =)




## Outputs

In [9]:
outputs_df = pd.DataFrame(outputs, columns = ['dataset', 'accuracy', 'f1', 'train_runtime'])
outputs_df

,dataset,accuracy,f1,train_runtime
0,human_enhancers_cohn,0.743811,0.745569,245.4339
1,human_ocr_ensembl,0.778267,0.780391,1242.0274
2,human_enhancers_ensembl,0.873135,0.873580,1061.0629


In [10]:
outputs_df.groupby('dataset').agg({'accuracy' : ['mean', 'sem'], 'f1' : ['mean','sem'], 'train_runtime': ['mean', 'sem']})

accuracy            f1     train_runtime    
                             mean sem      mean sem          mean sem
dataset                                                              
human_enhancers_cohn     0.743811 NaN  0.745569 NaN      245.4339 NaN
human_enhancers_ensembl  0.873135 NaN  0.873580 NaN     1061.0629 NaN
human_ocr_ensembl        0.778267 NaN  0.780391 NaN     1242.0274 NaN

In [11]:
# saving outputs to csv file
outputs_df.to_csv(OUTPUT_PATH, index=False)

In [12]:
outputs

[('human_enhancers_cohn', 0.743811168681635, 0.7455688965122927, 245.4339),
 ('human_ocr_ensembl', 0.7782673380636301, 0.7803910456219891, 1242.0274),
 ('human_enhancers_ensembl',
  0.8731352922182758,
  0.8735802310241643,
  1061.0629)]